In [ ]:
import pathlib as pl
import numpy as np
import matplotlib.pyplot as plt


import rasterio

import xarray as xa

import flopy

import pyflwdir

### Conversion factors

In [ ]:
ft2m = 1.0 / 3.28081
ft3tom3 = 1.0 * ft2m * ft2m * ft2m
ftpd2cmpy = 1000.0 * 365.25 * ft2m
mpd2cmpy = 100.0 * 365.25
mpd2inpy = 12.0 * 365.25 * 3.28081

### Load raster data

In [ ]:
nc_path = pl.Path("../data/synthetic_valley_truth.nc")
nc_ds = xa.open_dataset(nc_path)
nc_ds

In [ ]:
nc_ds["clay_kv"].plot()

In [ ]:
nc_ds["top_layer1"].plot()

In [ ]:
lake_location = flopy.utils.Raster.load("./data/lake_location_SI.tif")

In [ ]:
arr = nc_ds["k1_layer1"].to_numpy()
nlay = 5
shape3d = (nlay, arr.shape[0], arr.shape[1])

In [ ]:
def create_raster(src_path, dst_path, arr):
    with rasterio.open(src_path, "r") as src:
        if len(arr.shape) == 2:
            shape3d = (1, arr.shape[0], arr.shape[1])
            arr = arr.reshape(shape3d)
        nlay = arr.shape[0]

        # Copy the metadata profile from the source
        profile = src.profile

        # Update any specific profile parameters if needed (e.g., nodata value)
        profile.update(
            count=nlay,
            dtype=arr.dtype,
            nodata=-9999,
            compress="lzw",  # Add compression
        )

        # Open a new dataset in write mode ('w') and write the updated array
        with rasterio.open(dst_path, "w", **profile) as dst:
            for k in range(nlay):
                dst.write(arr[k], k + 1)

In [ ]:
k1 = np.zeros(shape3d, dtype=float)
for k in range(nlay):
    tag = f"k1_layer{k + 1}"
    k1[k, :, :] = nc_ds[tag].to_numpy() * ft2m
idx = np.isnan(k1[0, :, :])
k1[0, idx] = k1[1, idx]

create_raster("./data/lake_location_SI.tif", "./data/kaq_3d_SI.tif", k1)

In [ ]:
arr = nc_ds["clay_kv"].to_numpy() * ft2m
create_raster("./data/lake_location_SI.tif", "./data/k_clay_SI.tif", arr)

In [ ]:
arr = nc_ds["clay_location"].to_numpy()
arr[np.isnan(arr)] = 0
create_raster("./data/lake_location_SI.tif", "./data/clay_location_SI.tif", arr)

In [ ]:
arr = np.zeros(shape3d, dtype=float)
for k in range(nlay):
    tag = f"bottom_layer{k + 1}"
    arr[k, :, :] = nc_ds[tag].to_numpy() * ft2m
create_raster("./data/lake_location_SI.tif", "./data/bottom_3d_SI.tif", arr)

In [ ]:
with rasterio.open("./data/top_SI.tif", "r") as src:
    elevtn = src.read(1)
    nodata = src.nodata
    transform = src.transform

In [ ]:
flw = pyflwdir.from_dem(
    data=elevtn,
    transform=transform,
    latlon=False,
)
basins = flw.basins()

In [ ]:
im = plt.imshow(basins)
plt.colorbar(im, label="Basin ID")
plt.title("Flow Basins from DEM")
plt.xlabel("Column Index")
plt.ylabel("Row Index")

In [ ]:
arr = basins.copy().astype(float)
arr[basins < 4] = 0
arr[basins >= 4] = 1
plt.imshow(arr)
plt.title("Simplified Flow Basins")
plt.xlabel("Column Index")
plt.ylabel("Row Index")

In [ ]:
create_raster("./data/lake_location_SI.tif", "./data/runoff_routing_SI.tif", arr)

In [ ]:
nc_ds["runoff_routing"] = (("y", "x"), arr)

In [ ]:
temp_path = nc_path.parent / "temp.nc"
nc_ds.to_netcdf(temp_path)

In [ ]:
temp_path.rename(nc_path)